In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, Sequence
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, accuracy_score, roc_curve
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import random
import seaborn as sns
import cv2

In [2]:
class CustomDataset(Sequence):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return np.array(image), label

def get_data_loader(image_dir, batch_size, transform):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir)]
    labels = [1 if 'fractured' in img else 0 for img in os.listdir(image_dir)]
    dataset = CustomDataset(image_paths, labels, transform)
    return tf.data.Dataset.from_generator(
        lambda: dataset,
        output_signature=(
            tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    ).batch(batch_size)

In [3]:
base_dir = 'bone-frature/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification'
sub_dir = os.listdir(base_dir)
Categories = os.listdir(os.path.join(base_dir,"test"))

img_size = 100

Train_path = 'bone-frature/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
Test_path = 'bone-frature/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test'
Val_path = 'bone-frature/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val'

def fetch_data(path):
    data = []
    category = os.listdir(path)
    for cat in category:
        label = Categories.index(cat)
        img_dir = os.path.join(path,cat)
        images = os.listdir(img_dir)
        for img in images:
            try:
                img_array = cv2.imread(os.path.join(img_dir,img), cv2.IMREAD_GRAYSCALE)
                resized_img_array = cv2.resize(img_array, (img_size, img_size))
                data.append([resized_img_array,label])
            except Exception as e:
                pass
    return data


Training_data = fetch_data(Train_path)
Testing_data = fetch_data(Test_path)
Val_data = fetch_data(Val_path)

In [4]:
import random

random.shuffle(Training_data)

def makeX_y(data):
    makeX = []
    makeY = []
    
    for X, y in data:
        makeX.append(X)
        makeY.append(y)
        
    makeX = np.array(makeX).reshape(-1, img_size,img_size, 1)
    makeX = makeX / 255.0
    makeY = np.array(makeY)
    
    return makeX, makeY


X_train, y_train = makeX_y(Training_data)
X_test, y_test = makeX_y(Testing_data)
X_val, y_val = makeX_y(Val_data)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

# Create the model
Model = Sequential()

# Add the first convolutional layer
Model.add(Conv2D(64, (3, 3), padding='same', input_shape=X_train.shape[1:]))
Model.add(Activation('relu'))
Model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the second convolutional layer
Model.add(Conv2D(32, (3, 3), padding='same'))
Model.add(Activation('relu'))
Model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
Model.add(Flatten())

# Add the dense layers
Model.add(Dense(64))
Model.add(Activation('relu'))
Model.add(Dense(1))
Model.add(Activation('sigmoid'))

# Print the model summary
Model.summary()

C:\Users\pieri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 100, 100, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100, 100, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 50, 50, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,280,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,299,233 (4.96 MB)

 Trainable params: 1,299,233 (4.96 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
Model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

History = Model.fit(X_train, y_train, epochs = 15, batch_size = 32, validation_split=0.2)

Epoch 1/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 22s 92ms/step - accuracy: 0.7483 - loss: 0.4839 - val_accuracy: 0.9189 - val_loss: 0.2274
Epoch 2/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 92ms/step - accuracy: 0.9340 - loss: 0.1808 - val_accuracy: 0.9567 - val_loss: 0.1297
Epoch 3/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.9846 - loss: 0.0514 - val_accuracy: 0.9751 - val_loss: 0.0909
Epoch 4/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.9963 - loss: 0.0159 - val_accuracy: 0.9876 - val_loss: 0.0494
Epoch 5/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.9985 - loss: 0.0074 - val_accuracy: 0.9854 - val_loss: 0.0513
Epoch 6/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.9986 - loss: 0.0043 - val_accuracy: 0.9870 - val_loss: 0.0536
Epoch 7/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.9998 - loss: 0.0017 - val_accuracy: 0.9854 - val_loss: 0.0609
Epoch 8/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 21s 92ms/step - accuracy: 0.9991 - loss: 0.0041 - 

In [7]:
Validation_loss, Validation_accuracy = Model.evaluate(X_val, y_val)
print("Validation Loss - ", Validation_loss, "Validation Accuracy - ", Validation_accuracy)

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9820 - loss: 0.1455
Validation Loss -  0.17148765921592712 Validation Accuracy -  0.9758745431900024


In [8]:
y_pred = Model.predict(X_test)


threshold = 0.5
y_pred_label = []
fractured_count = 0
not_fractured_count = 0

for pred in y_pred:
    if(pred < 0.5):
        y_pred_label.append(0)
        not_fractured_count += 0
    else:
        y_pred_label.append(1)
        fractured_count += 1
        
true_count = 0
for true, pred in zip(y_test, y_pred_label):
    if true == pred:
        true_count += 1
print(f"Out of {len(y_test)} Samples, {true_count} Samples have the correct prediction from the model")
print("Accuracy ----> ", (true_count/len(y_test))*100)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Out of 506 Samples, 497 Samples have the correct prediction from the model
Accuracy ---->  98.22134387351778


In [9]:
Model.save('model_converted_new_final.keras')